In [2]:
import pandas as pd
import datetime

In [ ]:

# Function to get day of the week
def get_day_of_week_local(day, month, year):
    date = datetime.date(year, month, day)
    return date.strftime("%A")


In [ ]:
# Load the dataset
file_path = r'C:\Users\Lenovo\Desktop\DSA210_raw_Data\final_categorized_transactions.csv'
df = pd.read_csv(file_path)

# Add a new column for the day of the week
df['dayOfWeek'] = None

def calculate_day_of_week(row):
    try:
        date = row['Tarih'][0:10]  # Assuming 'Tarih' is in YYYY-MM-DD format
        year, month, day = map(int, date.split('-'))
        return get_day_of_week_local(day, month, year)
    except Exception as e:
        print(f"Error processing row with date {row['Tarih']}: {e}")
        return None

# Apply the function to the dataframe
df['dayOfWeek'] = df.apply(calculate_day_of_week, axis=1)

# Save the updated dataset
output_file_path = r'C:\Users\Lenovo\Desktop\DSA210_raw_Data\final_categorized_dayOfWeek.csv'
df.to_csv(output_file_path, index=False)

print("Processing complete. The updated file has been saved.")

In [6]:
# Load the dataset
file_path = r'C:\Users\Lenovo\Desktop\DSA210_raw_Data\final_categorized_dayOfWeek.csv'
df = pd.read_csv(file_path)

# Convert 'Tarih' to date only (strip time)
df['Tarih'] = pd.to_datetime(df['Tarih']).dt.date

df['Tarih'] = pd.to_datetime(df['Tarih']).dt.date

# Ensure 'Tutar' is properly converted to numeric
df['Tutar'] = pd.to_numeric(df['Tutar'], errors='coerce')

# Group by 'Tarih' (date only) and 'dayOfWeek' and calculate the required metrics
result = df.groupby(['Tarih', 'dayOfWeek']).apply(
    lambda x: pd.Series({
        'Total Spending': x.loc[x['Tutar'] < 0, 'Tutar'].sum(),
        'Total Income': x.loc[x['Tutar'] > 0, 'Tutar'].sum(),
        'Discretionary Amount': x.loc[(x['Tutar'] < 0) & (x['General Category'] == 'Discretionary'), 'Tutar'].sum(),
        'Essential Amount': x.loc[(x['Tutar'] < 0) & (x['General Category'] == 'Essential'), 'Tutar'].sum(),
        'Share of Discretionary': (
            x.loc[(x['Tutar'] < 0) & (x['General Category'] == 'Discretionary'), 'Tutar'].sum() /
            (
                x.loc[(x['Tutar'] < 0) & (x['General Category'] == 'Discretionary'), 'Tutar'].sum() +
                x.loc[(x['Tutar'] < 0) & (x['General Category'] == 'Essential'), 'Tutar'].sum()
            )
            if (
                x.loc[(x['Tutar'] < 0) & (x['General Category'] == 'Discretionary'), 'Tutar'].sum() +
                x.loc[(x['Tutar'] < 0) & (x['General Category'] == 'Essential'), 'Tutar'].sum()
            ) != 0 else 0
        )
    })
).reset_index()

# Save the summarized dataset to a new CSV file
output_file_path = r'C:\Users\Lenovo\Desktop\DSA210_raw_Data\final_categorized_merged.csv'
result.to_csv(output_file_path, index=False)

print("Summarized dataset saved successfully!")

Summarized dataset saved successfully!
